# SmartBeds - Preprocesado de One-Class

## Grado en Ingeniería Informática
## Universidad de Burgos
##### José Luis Garrido Labrador

In [1]:
import pandas as pd # se importa pandas como pd
import numpy as np  #numpy como np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from transformers import *
import pickle as pk
from utils import dibujado

### Carga de los datos

In [2]:
trozos_limpios = None
with open('../data/trozos_limpios.pkl','rb') as f:
    trozos_limpios = pk.load(f)
print(len(trozos_limpios))

91


Extraemos los día que tienen crisis

In [3]:
dias_crisis = []
for tl in trozos_limpios:
    for index,row in tl.iterrows():
        if row['target']:
            dias_crisis.append(tl)
            break
len(dias_crisis)

2

In [4]:
crisis_analisis = dias_crisis[0]

Se considera el inicio de la crisis a las 2018-11-10 03:36:10 y el final a las 2018-11-10 03:40:37. Considerando el inicio cuando P1 comienza a valor 0 y terminando cuando P1 baja de 8

In [5]:
start = pd.to_datetime("2018-11-10 03:36:10")
end = pd.to_datetime("2018-11-10 03:40:37")

crisis_real = crisis_analisis.copy()
mask = (crisis_real['DateTime'] < start) | (crisis_real['DateTime'] > end)
crisis_real.loc[mask,'target']=False

In [6]:
with open('crisis_18-nov-10-raw.pdd','wb') as fil:
    pk.dump(crisis_real,fil)

### Operaciones sobre los datos

In [7]:
svg = SavgolTransformer(15)
btr = ButterTransformer(3,0.05)
nor = Normalizer(max_=100)
avg = PipelineTransformer(StatisticsTransformer(mode='mean',window=25),nor)
std = PipelineTransformer(StatisticsTransformer(mode='std',window=25),nor)
ran = PipelineTransformer(StatisticsTransformer(mode='range',window=25),nor)
var = VarianceThresholdPD(threshold=0.5)
con = ConcatenateTransformer(avg,std,ran)

pps = PipelineTransformer(svg,con,var)
ppb = PipelineTransformer(btr,con,var)
ppr = PipelineTransformer(con,var)
ppr_s = PipelineTransformer(svg,var)
ppr_b = PipelineTransformer(btr,var)

In [8]:
datetimes = crisis_real['DateTime']
targets = crisis_real['target']
data = crisis_real.iloc[:,1:len(crisis_real.columns)-1]
data.head()

,P1,P2,P3,P4,P5,P6
3849685,4.098361,18.852459,22.131148,22.131148,13.114754,6.557377
3849686,4.098361,18.852459,21.311475,22.131148,13.114754,6.557377
3849687,4.918033,19.672131,22.131148,22.950820,13.934426,7.377049
3849688,4.098361,18.852459,21.311475,22.131148,13.114754,6.557377
3849689,0.000000,18.852459,22.131148,22.950820,13.114754,6.557377


In [9]:
lsavgol = ppr_s.fit_transform(data)
butterworth = ppr_b.fit_transform(data)
stats = ppr.fit_transform(data)
savgol_stats = pps.fit_transform(data)
butter_stats = ppb.fit_transform(data)


/usr/local/anaconda3/envs/TFG/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/usr/local/anaconda3/envs/TFG/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/usr/local/anaconda3/envs/TFG/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this w

In [14]:
name = "crisis_18-nov-10-"
names = ['savgol','butter','stats','savgol-stats','butter-stats']
datos = [lsavgol,butterworth,stats,savgol_stats,butter_stats]
for i in range(len(datos)):
    with open(name+names[i]+".pdd",'wb') as fil:
        pk.dump(pd.concat((datetimes,datos[i],targets),axis=1),fil)

In [15]:
with open(name+"stats.pdd",'rb') as f:
    opn = pk.load(f)
opn.head()

,DateTime,P1 mean 25,P2 mean 25,P3 mean 25,P4 mean 25,P5 mean 25,P6 mean 25,P1 std 25,P2 std 25,P3 std 25,P4 std 25,P5 std 25,P6 std 25,P1 range 25,P2 range 25,P3 range 25,P4 range 25,P5 range 25,P6 range 25,target
3849685,2018-11-09 21:03:27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3849686,2018-11-09 21:03:27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3849687,2018-11-09 21:03:28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3849688,2018-11-09 21:03:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3849689,2018-11-09 21:03:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
